In [19]:
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit
from scipy.stats import norm, stats

import sys
sys.path.append('/home/users/hswanson13/tbanalysis/') #stupid pytho
import run_analysis as ra
from tb_plotting import plotting_utilities as pu
from os import path
from datetime import datetime


In [20]:
import importlib
importlib.reload(ra)

def linear(x, m, b):
    #m, b = p0
    return m*x + b

def get_x(m, b, y):
    return (y - b) / m

def split_edges(t, v, gap):
    def get_split_indices(array, gap):
        indices = []
        last_index = 0
        for i in range(1, len(array)):
            if array[i] - array[i-1] > gap: 
                indices.append((last_index, i))
                last_index = i
        indices.append((last_index, len(array)))
        return indices
    edge_idx = get_split_indices(t, gap)
    return [t[start:end] for start, end in edge_idx], [v[start:end] for start, end in edge_idx]

def calc_rise_diff(t, v, edg_thrsh_low=0.08, edg_thrsh_high=0.875, meas_thrsh=0.4, second_rising_edg=True, cl_compare=None, make_plot=False):
    v_max = np.max(v)
    v_measure = v_max * meas_thrsh
    v_edg_low, v_edg_high = v_max*edg_thrsh_low, v_max*edg_thrsh_high

    edg_sel = (v > v_edg_low) & (v < v_edg_high)
    t_all_edgs, v_all_edgs = split_edges(t[edg_sel], v[edg_sel], 2)

    if make_plot:
        fig, ax = plt.subplots(1,1, figsize=(16,8))
        ax.scatter(t, v, color='black')
        ax.scatter(ak.flatten(t_all_edgs), ak.flatten(v_all_edgs), color='purple')
        ax.axhline(v_edg_low, c='purple', label=f"High Thresh = {edg_thrsh_high}")
        ax.axhline(v_edg_high, c='purple', label=f"Low Thresh = {edg_thrsh_low}")
    
    # if len(t_all_edgs) != 4:
    #     if make_plot:
    #         print(f"NOT 4 EDGES: {len(t_all_edgs)} with clock compare = {cl_compare:.5}")
    #         ax.set_title(f"FAILED FIT, clock compare = {cl_compare:.5}")
    #         if cl_compare:
    #             ax.axvline(cl_compare, linestyle="--", color="orange", label=f"Other Clock Val = {cl_compare:.5} ns")
    #         fig.show()
    #     return None
    
    rising_t_stamps = []
    frst_edg_falling = False
    for i_edg in range(len(t_all_edgs)):
        t_edg_arr = ak.to_numpy(t_all_edgs[i_edg])
        v_edg_arr = ak.to_numpy(v_all_edgs[i_edg])
        if len(t_edg_arr) < 2:
            #if only one point continue
            if i_edg == 0:
                frst_edg_falling = True
            continue
        coeff, pcov = curve_fit(linear, t_edg_arr, v_edg_arr)
        m, b = coeff
        #potentially check if the first i_edg == 0 slope is negative! meaning starting with falling edge no rising edge.
        if i_edg == 0 and m < 0:
            frst_edg_falling = True #doesnt 100% solve the issue but close enough ~0.2% of failure
        if m > 0:
            rising_t_stamps.append(get_x(m, b, v_measure))
            if make_plot:
                #ax.plot(t_edg_arr, m*t_edg_arr+b)
                ax.plot((v_edg_arr - b)/m, v_edg_arr, color="green") #have to do it like this, better for vertical lines

    if len(rising_t_stamps) < 2:
        #only one rising edge (only gets appended if m>0)
        if make_plot:
            print(f"ONLY ONE RISING EDGE: {len(rising_t_stamps)} with clock compare = {cl_compare:.5}")
        return None
    
    if second_rising_edg and not frst_edg_falling:
        clock = rising_t_stamps[1]
    else:
        clock = rising_t_stamps[0]
    T = rising_t_stamps[1] - rising_t_stamps[0]
    if make_plot:
        ax.axhline(v_measure, linestyle="--", color="red", label=f"Period @ {meas_thrsh*100}% Max V, T = {T:.5} ns")
        ax.axvline(clock, linestyle="--", color="blue", label=f"My Clock = {clock:.5} ns")
        if cl_compare is not None:
            ax.axvline(cl_compare, linestyle="--", color="orange", label=f"Compared Clock Val = {cl_compare:.5} ns")
        ax.set_title(f"Clock Waveform, T = {T:.5} ns")
        ax.set_xlabel("time (ns)")
        ax.set_ylabel("Clock Voltage")
        ax.legend(loc=1, prop={'size': 12}, frameon=True,fancybox=True)
        fig.show()
    return (clock, T)

def add_clock(events, amp_fraction=20, plot_num=None):
    def two_pnt_fit(times, clocks, edg_idx, amp):
        frst_times = times[:,edg_idx,0]
        scnd_times = times[:,edg_idx,1]

        frst_clocks = clocks[:,edg_idx,0]
        scnd_clocks = clocks[:,edg_idx,1]

        t_run = scnd_times - frst_times
        c_rise = scnd_clocks - frst_clocks
        slope = c_rise / t_run
        ybias = frst_clocks - slope*frst_times
        # calculate 20% of the amplitude
        #amp = (minima + np.abs(minima - maxima)*amp_fraction/100)[:,0]
        c_timestamp = np.array((amp[:,0] - ybias) / slope)
        return c_timestamp

    channel = events['channel']
    time = events['time']
    clocks = channel[:,2] # Hardcoded clock channel
    times = np.array(time[:,0])*10**9
    clock = np.array(clocks)

    if plot_num is not None:
        fig, ax = plt.subplots(1,1, figsize=(16,8))
        ax.scatter(times[plot_num], clock[plot_num], color='black')

    minima = np.tile(np.min(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    maxima = np.tile(np.max(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    amp = minima + np.abs(minima - maxima)*amp_fraction/100

    min_scale = np.abs(maxima - minima)/10.0

    clock_diff = np.diff(clock, append=0)
    rising_edge_mask = clock_diff > min_scale #to get all rising edges min_scale is a positive number

    all_rising_times = np.where(rising_edge_mask, times, 0)
    all_rising_clocks = np.where(rising_edge_mask, clock, 0)
    all_rising_times = ak.Array([sublist[sublist != 0] for sublist in all_rising_times])
    all_rising_clocks = ak.Array([sublist[sublist != 0] for sublist in all_rising_clocks])

    t_diffs = [np.diff(t_e) for t_e in all_rising_times]
    cut_points = [np.where(diff > 5)[0]+1 for diff in t_diffs]
    
    split_times = [np.split(t_e, ct_p) for t_e, ct_p in zip(all_rising_times, cut_points)]
    split_clocks = [np.split(t_e, ct_p) for t_e, ct_p in zip(all_rising_clocks, cut_points)]
    
    nmask = ak.num(split_times, axis=2) > 2 #keep only edges with more than 2 points point, doing keeping one than mask on biggest points is best practice lets try this tho

    rising_times = ak.Array(split_times)[nmask]
    rising_clocks = ak.Array(split_clocks)[nmask]

    #one could maybe do another mask that only keeps two arrays with the most points

    if plot_num is not None:
        ax.scatter(all_rising_times[plot_num], all_rising_clocks[plot_num], color='blue')

        ax.scatter(rising_times[plot_num][0], rising_clocks[plot_num][0], color='orange', label="firt rising edge")
        ax.scatter(rising_times[plot_num][1], rising_clocks[plot_num][1], color='pink', label="second rising edge")

        ax.legend()
        fig.show()
    
    clock_edge_0 = two_pnt_fit(rising_times, rising_clocks, 0, amp)
    clock_edge_1 = two_pnt_fit(rising_times, rising_clocks, 1, amp)

    Ts = clock_edge_1 - clock_edge_0

    return Ts

#print(np.std(gc_clocks))

# cntr = 0
# estrt, estop = 0, 10000
# fail_cntr = 0
# for events in uproot.iterate([input_data_path], step_size=10000):
    
#     for gc, event in zip(gc_clocks, events):
#         if estrt <= cntr < estop:
#             t = event.time[0] * 10**9
#             v = event.channel[2]

#             timing = calc_rise_diff(
#                 t, v, 
#                 edg_thrsh_low = 0.10,
#                 edg_thrsh_high = 0.90,
#                 meas_thrsh=0.20,
#                 cl_compare = gc,
#                 make_plot=False
#             )
            
#             if timing is None or abs(timing[0] - gc) > 0.1:
#                 fail_cntr += 1
#                 print(f"gc when mine fails! {gc}, {fail_cntr=}")
#                 # timing = calc_rise_diff(
#                 #     t, v, 
#                 #     edg_thrsh_low = 0.10,
#                 #     edg_thrsh_high = 0.90,
#                 #     meas_thrsh=0.20,
#                 #     cl_compare = gc,
#                 #     make_plot=True
#                 # )
                

#         if cntr == estop:
#             break
#         cntr+=1
#     break


In [ ]:

#telescope_data_path = "/ceph/cms/store/user/azecchin/DESY_ETROC_telescope_merged/ETROC_Telescope_data/Run_5050_5200_0.root"
input_data_path = "/ceph/cms/store/user/azecchin/DESY_ETROC_telescope_merged/ETROC_Telescope_data/Run_5050_5200_0.root"

# run_file = uproot.open(input_data_path)
# event_num = 10
# tree = run_file['pulse']
# event = tree.arrays(entry_start=event_num,entry_stop=event_num+1)

# gc_clocks = add_clock(tree, amp_fraction=20)

run_files = [input_data_path]

sel_scope_low = float(50) 
sel_scope_high = float(300)
sel_toa_low = float(20)
sel_toa_high = float(600)
sel_tot_low = float(70)
sel_tot_high = float(130)
sel_chi2_high = 50


print(f"GETTING PIXEL CENTERS FROM TRACKS")
xhits = np.empty((16,16), dtype=object)
yhits = np.empty((16,16), dtype=object)
for chunk in ra.run_chunker(run_files):
    events = uproot.concatenate(chunk)
    chi2_sel = events.chi2 < 50
    bad_track = ((events['x'] > -500) | (events['y'] > -500))
    for row in range(16):
        for col in range(16):
            pix_sel = ak.any(((events.row==row)&(events.col==col)), axis=1) #ak.any to grab track for event that had a hit in the row or col
            xtracks = ak.to_numpy(events[pix_sel & bad_track & chi2_sel].x)
            ytracks = ak.to_numpy(events[pix_sel & bad_track & chi2_sel].y)
            xhits[row,col] = np.append(xhits[row,col], xtracks) if xhits[row, col] is not None else xtracks
            yhits[row,col] = np.append(yhits[row,col], ytracks) if yhits[row, col] is not None else ytracks

#calculate the centers
pix_track_map = np.empty((16,16), dtype=object)
for row in range(16):
    for col in range(16):
        #print(row, col, xhits[row,col], yhits[row,col])
        pix_track_map[row,col] = ra.find_pxl_from_traks(xhits[row, col], yhits[row, col], eps=0.23, min_samples=10)


pu.plot_tracker_hits_per_pixel(xhits, yhits, pix_track_map, 9, 8)

print('px_track_done')

cal_vals = np.empty((16,16),dtype=object)
hit_matrix = np.zeros((16,16)) 
for chunk in ra.run_chunker(run_files):
    events = uproot.concatenate(chunk)
    scope_sel = ((events.amp[:,1] > sel_scope_low) & (events.amp[:,1] < sel_scope_high))
    for row in range(16):
        for col in range(16):
            pixel_sel = (events.row==row)&(events.col==col)
            total_mask = pixel_sel & scope_sel
            pt_mask = ra.px_track_mask(events, pix_track_map, row, col)
            #above returns None if there were no tracks in that pixel
            if pt_mask is not None:
                total_mask = total_mask & pt_mask

            selected_cals = ak.flatten(events.cal_code[total_mask])                        
            cal_vals[row][col] = np.append(cal_vals[row][col], selected_cals) if cal_vals[row, col] is not None else selected_cals
            hit_matrix[row][col] += ak.count(events.cal_code[total_mask])


cal_mode = np.zeros((16,16))
for row in range(16):
    for col in range(16):
        if len(cal_vals[row][col]) != 0:
            cal_mode[row][col] = stats.mode(ak.to_numpy(cal_vals[row][col]))[0] #results vary, may or may not have to do just [0] or two [0]
        else:
            cal_mode[row][col] = -9999 


In [ ]:
row, col = 9, 8
px_periods = []
for events in uproot.iterate(input_data_path, step_size=5000):

    data_sel = ra.data_mask(events, scope_low=sel_scope_low, scope_high=sel_scope_high, toa_low=sel_toa_low, toa_high=sel_toa_high, tot_low=sel_tot_low, tot_high=sel_tot_high)
    pix_sel = (events.row==row) & (events.col==col)
    
    events["gc_clock"] = add_clock(events, amp_fraction=20)

    total_sel = pix_sel & data_sel #& (events.nhits == 1) #nhits == 1 so no events with noisy hits
    pt_mask = ra.px_track_mask(events, pix_track_map, row, col)
    #above returns None if there were no tracks in that pixel
    if pt_mask is not None:
        total_sel = total_sel & pt_mask

    empty_arrs = ak.num(total_sel) > 0

    total_sel = total_sel[empty_arrs]
    gc_clock = events.gc_clock[empty_arrs]

    px_periods += ak.to_list(gc_clock[ak.any(total_sel, axis=1)])


np.std(px_periods)*len(px_periods)**0.5

In [ ]:
def gen_windows(start, stop, step):
    c = 0
    windows = [(start, stop)]
    while stop > start:
        if c % 2 == 0:
            start += step
            windows.append((start, stop))
        else:
            stop -= step
            windows.append((start, stop))
        c+=1
    return windows

input_data_path = "/home/etl/Test_Stand/ETROC2_Test_Stand/ScopeHandler/ScopeData/LecroyMerged/run_5055.root"
run_files = [input_data_path]

#edge_windows = gen_windows(0.01, 0.97, 0.02)
edge_windows = [(0.1,0.9), (0.1,0.89), (0.1,0.88), (0.1,0.87), (0.1,0.86), (0.1,0.85), (0.1,0.84), (0.1,0.83), (0.1,0.82), (0.1,0.81), (0.1,0.80), (0.1, 0.75), (0.1, 0.7)]

periods = {}
clock_diff = {}
cntr = 0
start = datetime.now()
for events in uproot.iterate(run_files, step_size=10000):
    for event in events:
        t = event.time[0] * 10**9
        v = event.channel[2]
        for low, high in edge_windows:
            timing = calc_rise_diff(
                t, 
                v, 
                edg_thrsh_low = low,
                edg_thrsh_high = high,
                meas_thrsh = 0.5,
                second_rising_edg=False
            )
            if timing is not None:
                clock, T = timing
                if (low, high) in periods:
                    periods[(low, high)].append(T)
                    clock_diff[(low, high)].append(event.Clock - clock)
                else:
                    periods[(low, high)] = [T]
                    clock_diff[(low, high)] = [event.Clock - clock]
        if cntr == 1000:
            print(10000/cntr, ra.elapsed_time(start))
        cntr+=1
    break

In [ ]:
from tb_plotting import plotting_utilities as pu


fig, ax = plt.subplots(1,1, figsize=(18,14), layout="compressed")
#periods = pu.get_data("/home/users/hswanson13/tbanalyisis/analysis/investigations/clock_shape/periods.pkl")

jit_pos = np.arange(len(periods))
jit_labels = [f"({wind[0]:.4},{wind[1]:.4})" for wind in periods]
jit_vals = np.array([np.round(np.std(periods[wind])*1000, 2) for wind in periods])

jitter = ax.barh(jit_pos, jit_vals,align="center")
ax.bar_label(jitter, fontsize=12)


# eff_cut = 0.8
# jit_cut = jit_vals[eff_vals > eff_cut]

# min_jit_idx = np.where(jit_vals == np.min(jit_cut))[0][0]
# max_eff_idx = np.where(eff_vals == np.max(eff_vals))[0][0]
# print(max_eff_idx)
# axes[0].axhline(min_jit_idx, color="green", linestyle="--", label="Min Jitter > 0.8 Eff")
# axes[0].axhline(max_eff_idx, color="orange", linestyle="--", label="Highest Eff")

ax.set_yticks(jit_pos, labels=jit_labels)

ax.invert_yaxis()
ax.set_xlabel("jitter (ps)")
ax.set_ylabel("Threshold (low, high)")

fig.suptitle("Comparing Clock Jitter of Different Thresholds with it's Efficiency")

ax.tick_params(axis='y', labelsize=12)
ax.legend(loc='center')

plt.tight_layout()
plt.show()

#symmetric ?
#also play with threshold, try middle of line
#try using my clock vs theirs, take 40 at the time value, check if it is close to the ones they give in events.Clock

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(16,8), sharey=True, layout="compressed")
periods = pu.get_data("/home/users/hswanson13/tbanalyisis/analysis/investigations/clock_shape/periods.pkl")

jit_pos = np.arange(len(periods))
jit_labels = np.array([f"({wind[0]:.4},{wind[1]:.4})" for wind in periods])
jit_vals = np.array([np.round(np.std(periods[wind])*1000, 2) for wind in periods])
eff_vals = np.array([len(periods[wind])/10000 for wind in periods])

eff_cut = 0.8
jit_cut = jit_vals[eff_vals > eff_cut]
min_jit_idx = np.where(jit_vals == np.min(jit_cut))[0][0]
max_eff_idx = np.where(eff_vals == np.max(eff_vals))[0][0]

window_mask = jit_pos[(jit_pos >= min_jit_idx) & (jit_pos <= max_eff_idx)]

#apply mask for the area of interest
jit_pos = jit_pos[window_mask]
jit_vals = jit_vals[window_mask]
jit_labels = jit_labels[window_mask]
eff_vals = eff_vals[window_mask]

jitter = axes[0].barh(jit_pos, jit_vals, align="center")
axes[0].bar_label(jitter, fontsize=12)
eff = axes[1].barh(jit_pos, eff_vals, align="center", color="grey")
axes[1].bar_label(eff, fontsize=8)


axes[0].axhline(min_jit_idx, color="green", linestyle="--", label="Min Jitter > 0.8 Eff")
axes[0].axhline(max_eff_idx, color="orange", linestyle="--", label="Highest Eff")

axes[0].set_yticks(jit_pos, labels=jit_labels)
axes[1].set_yticks(jit_pos, labels=jit_labels)

axes[0].invert_yaxis()
axes[0].set_xlabel("jitter (ps)")
axes[0].set_ylabel("Threshold (low, high)")
axes[1].set_xlabel("efficiency")

fig.suptitle("Comparing Clock Jitter of Different Thresholds with it's Efficiency")

axes[0].tick_params(axis='y', labelsize=12, right=False)
axes[1].tick_params(axis='y', labelsize=12, right=False)

#axes[0].legend(loc='center')

plt.tight_layout()
plt.show()

#symmetric ?
#also play with threshold, try middle of line
#try using my clock vs theirs, take 40 at the time value, check if it is close to the ones they give in events.Clock

In [ ]:
#Comparing My Clock vs the One in the events

clock_diff = pu.get_data("/home/users/hswanson13/tbanalyisis/analysis/investigations/clock_shape/clock_diff.pkl")

chosen_cdiff = np.array(clock_diff[(0.11000000000000001, 0.8899999999999999)])

fig, ax = plt.subplots(1,1, figsize=(8,8), layout="compressed")


#chosen_cdiff = chosen_cdiff[chosen_cdiff < 15]
ax.hist(chosen_cdiff, 50)
ax.axvline(np.mean(chosen_cdiff), color="red", linestyle="--")


ax.set_xlabel("Event Clock - My Clock (ns)")
ax.tick_params(axis='y', labelsize=12)

plt.show()

In [ ]:
#Comparing My Clock vs the One in the events

clock_diff = pu.get_data("/home/users/hswanson13/tbanalyisis/analysis/investigations/clock_shape/clock_diff.pkl")

match_cntr = 0
for wind in clock_diff:
    chosen_cdiff = np.array(clock_diff[wind])
    chosen_cdiff = chosen_cdiff[chosen_cdiff < 15]

    cdiff_mean = np.mean(chosen_cdiff)
    cdiff_std = np.std(chosen_cdiff)

    #if (cdiff_mean + cdiff_std) > 0 or (cdiff_mean - cdiff_std ) < 0:
    if (cdiff_mean - 2*cdiff_std ) <= 0 <= (cdiff_mean + 2*cdiff_std):
        if match_cntr == 1:
            print(cdiff_mean + cdiff_std, cdiff_mean - cdiff_std)
            fig, ax = plt.subplots(1,1, figsize=(8,8), layout="compressed")

            ax.hist(chosen_cdiff, 50)
            ax.axvline(cdiff_mean, color="red", linestyle="--")
            ax.set_xlabel("Event Clock - My Clock (ns)")
            ax.set_ylabel("Counts")

            ax.tick_params(axis='y', labelsize=12)
            plt.title(f"Clock Difference for Config Low={wind[0]:.3}, High={wind[1]:.3}")
            plt.show()
            break
        match_cntr += 1

In [ ]:
#investigating why some are double peak

def gen_windows(start, stop, step):
    c = 0
    windows = [(start, stop)]
    while stop > start:
        if c % 2 == 0:
            start += step
            windows.append((start, stop))
        else:
            stop -= step
            windows.append((start, stop))
        c+=1
    return windows

telescope_data_path = "/ceph/cms/store/user/azecchin/DESY_ETROC_telescope_merged/ETROC_Telescope_data/"
run_files = [path.join(telescope_data_path, f"Run_5050_5200_{i}.root") for i in range(6+1)]


low, high = 0.03, 0.97
zr_cntr = 0
for events in uproot.iterate(run_files, step_size=10000):
    for event in events:
        t = event.time[0] * 10**9
        v = event.channel[2]
        timing = calc_rise_diff(
            t, 
            v, 
            edg_thrsh_low = low,
            edg_thrsh_high = high,
            meas_thrsh = 0.4,
        )
        if timing is not None:
            myclock, _ = timing
            cdiff = event.Clock - myclock

            if cdiff > 20:
                if zr_cntr == 3:
                    print(event.Clock)
                    print(event['LP2_20'][1] * 10**9)
                    timing = calc_rise_diff(
                        t, 
                        v, 
                        edg_thrsh_low = low,
                        edg_thrsh_high = high,
                        meas_thrsh = 0.4,
                        make_plot=True
                        )
                    break
                zr_cntr += 1
    break